In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import random as rm
import numpy as np
import pandas as pd
from numpy.random import default_rng
import time
from numba import njit

rng = default_rng()

In [3]:
def str2int_arr(str):
    return [float(i) for i in str]

def graph_from_file(file):
    w = []
    with open(file, "r") as f:
        for line in f:
            arr = line.split()
            map(int, arr)
            w.append(str2int_arr(arr))
    return nx.from_numpy_matrix(np.array(w))

def graph_from_dimacs(file):
    G = nx.Graph()
    with open(file, "r") as f:
        for line in f:
            buff = line.split()
            if buff[0]=="p":
                G.add_nodes_from([i for i in range(1, int(buff[2])+1 )])
                break
        for line in f:
            edge = line.split()
            G.add_edge(int(edge[1]), int(edge[2]))
    return G

In [4]:
def digits_from_01(arr):
    digits = []
    for i in range(len(arr)):
        if int(arr[i]) == 1:
            digits.append(i) 
    return digits
    

def initial_population(n, size):
    population = []
    for _ in range(size):
        individual = ""
        for _ in range(n):
            individual += rm.choice("0")
        population.append(individual)
    return population
    
def adaptability(G, individual, B=2):
    subgraph_v = np.array([], dtype="int")
    
    for i in range(individual.size):
        if individual[i]==1:
            subgraph_v = np.append(subgraph_v, [i])
            
    subgraph = nx.subgraph(G, list(subgraph_v))
    n = subgraph_v.size
    return max((-n*(n-1)//2 + subgraph.number_of_edges()) * B + n, 1) #max(n-(n*(n-1)//2 + subgraph.number_of_edges())*fine, 1)

def roulette(population, k):
    
    population_weights = [adaptability(G, i) for i in population]
    #if sum(population_weights)==0 :
    #    population_weights = [1 for _ in range(len(population_weights))]
    descendants = rm.choices(population, population_weights, k=k)
    
    return descendants

def crossover(parents):
    point = rm.randint(0, parents[0].size)
    child = np.concatenate((parents[0][:point], parents[1][point:]))
    return child 

def mutation(individual, probability = 0.5):
    if rm.random()>probability:
        return individual
    
    point = rm.randint(0, individual.size-1)
    
    individual[point] ^= 1
    
    return individual

def multy_mutation(individual):
    num_mutation=5
    n = individual.size
    for i in range(num_mutation):
        individual[np.random.randint(0, n)] ^= 1
    
    return individual

def print_p(population):
    for i in population:
        for j in i:
            print(j, end='')
        print()


def GA( p_size = 40,
        np_size = 40,
        max_iteration = 1500,
        elite_size = 5,):
    
    population = np.zeros((p_size, G.number_of_nodes()), dtype="byte")  # np.array(initial_population(n, p_size), dtype="str")
    population = np.array(list(map(multy_mutation, population)), dtype="byte")
    p_adaptability = np.array([adaptability(G, ind) for ind in population], dtype="int")

    new_population = np.zeros((np_size, G.number_of_nodes()), dtype="byte")
    new_p_adaptability = np.zeros(np_size, dtype="int")

    for _ in range(max_iteration):
        # выбор родителей + скрещивание + мутация
        for i in range(np_size):
            
            new_population[i] = mutation(crossover( rm.choices(population, p_adaptability, k=2) ), probability=0.6 )#  ???
            new_p_adaptability[i] = adaptability(G, new_population[i], B=1)

        # объединение популяций
        new_population = np.concatenate((population, new_population[:np_size]))
        new_p_adaptability = np.concatenate((p_adaptability, new_p_adaptability[:np_size]))
        
        # выбор новой популяции
        new_num = rm.choices(np.arange(p_size + np_size), new_p_adaptability, k=p_size)  # the numbers of individuals for the new population
        # выбор самых приспособленных
        elite_index = np.argpartition(-new_p_adaptability, elite_size)
        
        for i in range(elite_size):
            population[i] = new_population[elite_index[i]]
            p_adaptability[i] = new_p_adaptability[elite_index[i]]
        for i in range(elite_size, p_size):
            population[i] = new_population[new_num[i]]
            p_adaptability[i] = new_p_adaptability[new_num[i]]
        
    return p_adaptability, population

## Загрузка графа

In [5]:
G = graph_from_file("data/graph_examples/34_14")
#----------Hyperparameters----------
p_size = 20
np_size = 20
iterations = 1000
elite_size = 3
estimated_click_size = 14
#-----------#-----------#----------- 

#p_adaptability, population = GA(p_size, np_size, iterations, elite_size, estimated_click_size)
        
#print(p_adaptability)
#print_p(population)

## Таблица с результатами тестов

In [6]:
df = pd.DataFrame(columns=['population size', 'new population size', 'number of iterations', 'number of runs', 'best score', 'time'])

In [7]:
G = graph_from_dimacs("data/DIMACS_subset_ascii/C500.9.clq")
#----------Hyperparameters----------
p_size = 40
np_size = 40
iterations = 15 #00
elite_size = 5
estimated_click_size = 58
number_of_runs = 1 #5
#-----------#-----------#----------- 

In [8]:
def mult_GA(num_parameter, arr,  p_size, np_size, iterations, number_of_runs, df):
    print("aaaa")
    for i in arr:
        
        start_time = time.time()
        input = [p_size, np_size, iterations, number_of_runs]
        input[num_parameter] = i 
        result = 0
        
        for _ in range(input[3]):
            p_adaptability, population = GA(*input[:3], elite_size)
            result = max(result, p_adaptability[0])        

        df.loc[len(df)] = [*input, result, (time.time() - start_time)]

In [15]:
number_of_runs = 3

pop_sizes =  [10, 20]#, 50, 75, 100, 150]
mult_GA(0, pop_sizes, p_size, np_size, iterations, number_of_runs, df)

In [ ]:
np_sizes =  [10, 20]#, 50, 75, 100, 150]
mult_GA(1, np_sizes, p_size, np_size, iterations, number_of_runs, df)

In [ ]:
iterations_arr = [100]#, 250, 500, 1000, 2500, 5000, 10000]
mult_GA(2, iterations_arr, p_size, np_size, iterations, number_of_runs, df)

In [58]:
numbers_of_runs_arr = [1, 2]#, 5, 10, 15, 25, 50]
mult_GA(3, numbers_of_runs_arr, p_size, np_size, iterations, number_of_runs, df)

In [10]:
df

,population size,new population size,number of iterations,number of runs,best score,time


In [27]:
from multiprocessing import Process
import os

if __name__ == '__main__':
    pop_sizes =  [10, 20]
    p = Process(target=mult_GA, args=(0, pop_sizes, p_size, np_size, iterations, number_of_runs, df,))
    #mult_GA(0, pop_sizes, p_size, np_size, iterations, number_of_runs, df)
    p.start()
    p.join()

In [ ]:
df

,population size,new population size,number of iterations,number of runs,best score,time


In [19]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    #info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

## Генерация популяции для сети Хопфилда

In [60]:
p_adaptability, population = GA(p_size, np_size, iterations, elite_size)

In [62]:
population[:5]

array([[0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1]], dtype=int8)

In [70]:
import pickle
with open('data/graphFromGaToHop/g'+str(G.number_of_nodes())+"_"+str(p_adaptability[0])+'.pkl', 'wb') as out:
    pickle.dump(population, out, 2)